# Object Detection on Waste Recycling Plant Dataset

In this notebook you will:
- Train
- Validate
- Test

a yolov8 custom model

Source of dataset: https://www.kaggle.com/datasets/parohod/warp-waste-recycling-plant-dataset?select=.idea

# Train Model

#### Check acces to GPU

Let's make sure that we have access to GPU.

In [1]:
!nvidia-smi

Tue Sep 12 07:00:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Installation

In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.3/616.3 kB 5.4 MB/s eta 0:00:00


Import libraries for training and splitting data

In [3]:
import os
import shutil
import random

from tqdm.notebook import tqdm

Connect to google drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Split data into train and validation set

In [5]:
#create file paths for the training and validation sets
train_path_img = "/content/drive/MyDrive/Garbage_Detection/data/train/images/"
train_path_label = "/content/drive/MyDrive/Garbage_Detection/data/train/labels/"
val_path_img = "/content/drive/MyDrive/Garbage_Detection/data/val/images/"
val_path_label = "/content/drive/MyDrive/Garbage_Detection/data/val/labels/"

In [6]:
# Define a function for splitting training data to train and validation
def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    # split the data
    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories for the images and their labels

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### copying images and labels to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path[0:43] + 'labels/'  + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copyt images and labels to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path[0:43] + 'labels/' + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Validation data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

In [7]:
# split data
# Note: it may take 20mins to run
train_test_split('/content/drive/MyDrive/Garbage_Detection/unsampled_data/train_images/')

------ PROCESS STARTED -------
--- This folder has a total number of 2452 images---


  0%|          | 0/1962 [00:00<?, ?it/s]

------ Training data created with 80% split 1962 images -------


  0%|          | 0/490 [00:00<?, ?it/s]

------ Validation data created with a total of 490 images ----------
------ TASK COMPLETED -------


Create yaml file for training

In [8]:
import yaml

In [9]:
# infomation for yaml document
data = {
    'train': '/content/drive/MyDrive/Garbage_Detection/data/train/images/', # path to training images
    'val': '/content/drive/MyDrive/Garbage_Detection/data/val/images/', #path to validation images

    'nc': 28, # number of classes
    'names': ['bottle-blue', 'bottle-green', 'bottle-dark', 'bottle-milk', 'bottle-transp', 'bottle-multicolor', 'bottle-yogurt', 'bottle-oil', 'cans', 'juice-cardboard', 'milk-cardboard', 'detergent-color', 'detergent-transparent', 'detergent-box', 'canister', 'bottle-blue-full', 'bottle-transp-full', 'bottle-dark-full', 'bottle-green-full', 'bottle-multicolor-full', 'bottle-milk-full', 'bottle-oil-full', 'detergent-white', 'bottle-blue5l', 'bottle-blue5l-full', 'glass-transp', 'glass-dark', 'glass-green'] #class names
}

In [10]:
# create yaml file
with open("dataset.yaml" , "w") as f:
   yaml.dump(data,f,default_flow_style = False)

In [11]:
# save yaml file to google drive
!cp /content/dataset.yaml /content/drive/MyDrive/Garbage_Detection/dataset.yaml

Check environment

In [12]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.175 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.3/78.2 GB disk)


Train model

In [13]:
!yolo task=detect mode=train model=yolov8m.pt data=/content/drive/MyDrive/Garbage_Detection/dataset.yaml epochs=100 imgsz=640 batch=16 patience=20 project=/content/drive/MyDrive/Garbage_Detection/training_results/weights name=garbage_detection

# comment the code above and uncomment the code below to resume training if notebook is interupted
# Note: you will need to run all the previous lines of code

# !yolo task=detect mode=train resume data=/content/drive/MyDrive/Garbage_Detection/dataset.yaml model=/content/drive/MyDrive/Garbage_Detection/training_results/weights/last.pt

100% 49.7M/49.7M [00:00<00:00, 70.0MB/s]
Ultralytics YOLOv8.0.175 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/Garbage_Detection/dataset.yaml, epochs=100, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/Garbage_Detection/training_results/weights, name=garbage_detection, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width

## Validate model

Validate model on unseen data

Note: the test data will be used in this case. It will be split into 10 for visual testing and the rest for validation metrics

Define paths for validation images

In [14]:
#create file paths for the training and validation sets
validation_path_img = "/content/drive/MyDrive/Garbage_Detection/data/validation/images/"
validation_path_label = "/content/drive/MyDrive/Garbage_Detection/data/validation/labels/"
test_path_img = "/content/drive/MyDrive/Garbage_Detection/data/test/images/"
test_path_label = "/content/drive/MyDrive/Garbage_Detection/data/test/labels/"

In [15]:
# Define a function for splitting data into validation and test
def train_test_split(path,neg_path=None, split = 0.04):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    # split the data
    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories for the images and their labels

    os.makedirs(validation_path_img, exist_ok = True)
    os.makedirs(validation_path_label, exist_ok = True)
    os.makedirs(test_path_img, exist_ok = True)
    os.makedirs(test_path_label, exist_ok = True)


    ### copying images and labels to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{validation_path_img}/" + filex + '.jpg' )
      shutil.copy2(path[0:48] + 'labels/'  + filex + '.txt', f"{validation_path_label}/" + filex + '.txt')



    print(f"------ Validation data created with 2% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{validation_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Validation data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copy images and labels to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{test_path_img}/" + filex + '.jpg' )
      shutil.copy2(path[0:48] + 'labels/' + filex + '.txt', f"{test_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

In [16]:
# split data
train_test_split('/content/drive/MyDrive/Garbage_Detection/unsampled_data/test_images/')

------ PROCESS STARTED -------
--- This folder has a total number of 492 images---


  0%|          | 0/473 [00:00<?, ?it/s]

------ Validation data created with 2% split 473 images -------


  0%|          | 0/19 [00:00<?, ?it/s]

------ Testing data created with a total of 19 images ----------
------ TASK COMPLETED -------


Create yaml file for validation


In [17]:
import yaml

In [18]:
# infomation for yaml document
data_validation = {
    'train': '/content/drive/MyDrive/Garbage_Detection/data/validation/images/', # path to training images
    'val': '/content/drive/MyDrive/Garbage_Detection/data/test/images/', #path to validation images

    'nc': 28, # number of classes
    'names': ['bottle-blue', 'bottle-green', 'bottle-dark', 'bottle-milk', 'bottle-transp', 'bottle-multicolor', 'bottle-yogurt', 'bottle-oil', 'cans', 'juice-cardboard', 'milk-cardboard', 'detergent-color', 'detergent-transparent', 'detergent-box', 'canister', 'bottle-blue-full', 'bottle-transp-full', 'bottle-dark-full', 'bottle-green-full', 'bottle-multicolor-full', 'bottle-milk-full', 'bottle-oil-full', 'detergent-white', 'bottle-blue5l', 'bottle-blue5l-full', 'glass-transp', 'glass-dark', 'glass-green'] #class names
}

In [19]:
# create yaml file
with open("dataset_validation.yaml" , "w") as f:
   yaml.dump(data_validation,f,default_flow_style = False)

In [20]:
# save yaml file to google drive
!cp /content/dataset_validation.yaml /content/drive/MyDrive/Garbage_Detection/dataset_validation.yaml

Validate trained model

Graphs and metrics will be produced

In [21]:
!yolo task=detect mode=val model=/content/drive/MyDrive/Garbage_Detection/training_results/weights/garbage_detection/weights/best.pt data=/content/drive/MyDrive/Garbage_Detection/dataset_validation.yaml

Ultralytics YOLOv8.0.175 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25855972 parameters, 0 gradients
val: Scanning /content/drive/MyDrive/Garbage_Detection/data/test/labels... 19 images, 0 backgrounds, 0 corrupt: 100% 19/19 [00:00<00:00, 268.97it/s]
val: New cache created: /content/drive/MyDrive/Garbage_Detection/data/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 2/2 [00:01<00:00,  1.01it/s]
                   all         19         73      0.613      0.562      0.606      0.504
           bottle-blue         19          4      0.445       0.75      0.731      0.618
          bottle-green         19          5      0.563      0.778      0.585      0.543
           bottle-dark         19          7      0.555      0.857      0.884      0.726
         bottle-transp         19         13      0.638      0.385      0.461      0.272
     bottle-multicolor         19     

In [22]:
# save validation results to a folder in your Drive with the name validation
!cp -r /content/runs/detect/val /content/drive/MyDrive/Garbage_Detection/validation

## Testing

Test using the 10 test images

In [26]:
# test using the best weights from the training_results folder
!yolo task=detect mode=predict model=/content/drive/MyDrive/Garbage_Detection/training_results/weights/garbage_detection/weights/best.pt conf=0.30 source=/content/drive/MyDrive/Garbage_Detection/data/sample_test/ save=True

Ultralytics YOLOv8.0.175 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25855972 parameters, 0 gradients

image 1/8 /content/drive/MyDrive/Garbage_Detection/data/sample_test/POSAD_1_12-Sep_12-22-21.jpg: 384x640 1 bottle-milk, 1 bottle-oil, 1 canister, 1 bottle-green-full, 119.9ms
image 2/8 /content/drive/MyDrive/Garbage_Detection/data/sample_test/POSAD_1_12-Sep_16-29-25.jpg: 384x640 1 bottle-blue, 1 bottle-dark, 1 bottle-milk, 1 bottle-oil, 1 cans, 1 canister, 1 bottle-transp-full, 1 bottle-dark-full, 1 bottle-milk-full, 1 bottle-blue5l, 25.9ms
image 3/8 /content/drive/MyDrive/Garbage_Detection/data/sample_test/POSAD_1_13-Sep_05-54-04.jpg: 384x640 1 bottle-blue, 1 bottle-milk, 1 bottle-transp, 1 bottle-oil, 1 detergent-white, 1 bottle-blue5l, 25.9ms
image 4/8 /content/drive/MyDrive/Garbage_Detection/data/sample_test/Robo_25-Mar_15-47-34.jpg: 384x640 1 bottle-green, 1 bottle-transp-full, 1 bottle-green-full, 1 bottle-milk-full, 2 bottle

In [27]:
# save prediction results to output folder in google drive
!cp -r /content/runs/detect/predict2 /content/drive/MyDrive/Garbage_Detection/FinalPrediction/